# 🐷💫 Susie Package Astroplan Schedule Generation Example

## 🔵 Import the necessary python libraries and Susie objects.

Importing any packages and libraries needed, (Susie package coming directly from filesystem)

In [1]:
import sys
sys.path.append("..")
import pytz
import numpy as np
import pandas as pd
from src.susie.ephemeris import Ephemeris
from src.susie.timing_data import TimingData
import matplotlib.pyplot as plt

import astropy.units as u
from astropy.time import Time
from astroplan import EclipsingSystem, AtNightConstraint, AltitudeConstraint, is_event_observable

## 🔵 Import Data

Download the timing data (that includes occultations) from the GitHub repository

In [2]:
url = 'https://raw.githubusercontent.com/BoiseStatePlanetary/susie/main/example_data/wasp12b_tra_occ.csv'

# Read the CSV file directly from the URL
data = pd.read_csv(url)
tra_or_occs = np.array(data["tra_or_occ"])
epochs = np.array(data["epoch"].astype('int'))
mid_times = np.array(data["mid_time"])
mid_time_errs = np.array(data["mid_time_err"])

## 🔵 The Package

### 🔷 **STEP 1:** Add your transit and occultation data to the TimingData object.

In [3]:
# Create new transit times object with above data
timing_obj = TimingData('jd', epochs, mid_times, mid_time_uncertainties=mid_time_errs, tra_or_occ=tra_or_occs, time_scale='tdb')

### 🔷 **STEP 2:** Create the Ephemeris object and add your TimingData object.

In [4]:
ephemeris_obj = Ephemeris(timing_obj)

We'll use the quadratic model for this example. Get the model data dictionary from the ephemeris object.

In [5]:
quad_model_data = ephemeris_obj.get_model_ephemeris("quadratic")

Now we need two main objects to get the observing schedule
 1. The Observer object, which is created using the observer's Earth location (with longitude, latitude, and elevation if known)
 2. The Target object, which is created using the star's sky coordinates (with either the star's name or the right ascension and declination)

In [6]:
# Create observer object (at Boise State)
observer_obj = ephemeris_obj.create_observer_obj(timezone="US/Mountain", longitude=-116.208710, latitude=43.602,
                                                    elevation=821, name="BoiseState")

In [7]:
# Create target object (WASP-12)
target_obj = ephemeris_obj.create_target_obj("WASP-12")

Now we can call get observing schedule. For this we will need
   1. Model data dictionary from ouru desired ephemeris (we will use the quadratic ephemeris that we created above)
   2. A timezone (we will use US/Mountain for Boise State time)
   3. The observer object (created above)
   4. The target object (created above)
   5. The number of transits we want in our schedule
   6. The number of occultations we want in our schedule
   7. The observation start time, which will be the moment we start looking for observable transits/occultations
   8. And one of the two:
      - The name of the exoplanet, which will be used to get an eclipse duration OR
      - The eclipse duration itself if the object is either not a known exoplanet or you want to customize this parameter

In [14]:
ephemeris_obj.get_observing_schedule(quad_model_data, "US/Mountain", observer_obj, target_obj, 25, 0, "2024-01-07", "WASP-12 b", csv_filename="../example_data/schedule_test.csv")

,ingress,egress,type
0,2024-01-15 20:17:16 -0700,2024-01-15 23:17:19 -0700,transit
1,2024-01-16 22:28:54 -0700,2024-01-17 01:28:58 -0700,transit
2,2024-01-18 00:40:33 -0700,2024-01-18 03:40:37 -0700,transit
3,2024-01-27 20:25:21 -0700,2024-01-27 23:25:25 -0700,transit
4,2024-01-28 22:37:00 -0700,2024-01-29 01:37:04 -0700,transit


In [9]:
eclipse_duration = ephemeris_obj._get_eclipse_duration("WASP-12 b")

In [10]:
eclipse_duration

<MaskedQuantity 3.001 h>

In [11]:
# Grab the most recent mid transit time
primary_eclipse_time = Time(ephemeris_obj.timing_data.mid_times[-1], format='jd')
# Pull orbital period from the model
orbital_period = quad_model_data['period'] * u.day
# eclipse_duration = ephemeris_obj._get_eclipse_duration("WASP-12 b")
# Create EclipsingSystem object
eclipsing_system = EclipsingSystem(primary_eclipse_time=primary_eclipse_time,
                        orbital_period=orbital_period, duration=eclipse_duration)
# Set the observational parameters
# Time to start looking
obs_time = Time("2024-01-07 00:00")
# Grab the number of transits and occultations asked for
ing_egr_transits = eclipsing_system.next_primary_ingress_egress_time(obs_time, n_eclipses=10)
ing_egr_occultations = eclipsing_system.next_secondary_ingress_egress_time(obs_time, n_eclipses=10)
# We need to check if the events are observable
constraints = [AtNightConstraint.twilight_civil(), AltitudeConstraint(min=30*u.deg)]
transits_bool = is_event_observable(constraints, observer_obj, target_obj, times_ingress_egress=ing_egr_transits)
occultations_bool = is_event_observable(constraints, observer_obj, target_obj, times_ingress_egress=ing_egr_occultations)
observable_transits = ing_egr_transits[transits_bool[0]]
observable_occultations = ing_egr_occultations[occultations_bool[0]]
# Change the returned observation times to given timezone
tz = pytz.timezone("US/Mountain")
converted_transits = observable_transits.to_datetime(timezone=tz)
converted_occultations = observable_occultations.to_datetime(timezone=tz)
# Create dataframe from this
transit_df = pd.DataFrame(converted_transits)
occultation_df = pd.DataFrame(converted_occultations)
transit_df = transit_df.map(lambda dt: dt.strftime("%Y-%m-%d %H:%M:%S %z"))
transit_df = transit_df.rename(columns={0: "ingress", 1: "egress"})
occultation_df = occultation_df.map(lambda dt: dt.strftime("%Y-%m-%d %H:%M:%S %z"))
occultation_df = occultation_df.rename(columns={0: "ingress", 1: "egress"})
transit_df["type"] = "transit"
occultation_df["type"] = "occultation"
final_df = pd.concat([transit_df, occultation_df], ignore_index=True)
sorted_df = final_df.sort_values(by="ingress", ascending=True)
sorted_df.to_csv("../example_data/schedule_test.csv", index=False)